## 1D Schroedinger Equation Solver - Holoviews

by Stan Mohler, Jr.  10/2022

This Notebook numerically solves the 1-D time-dependent Schroedinger
equation for a free particle with periodic (wrap-around) boundary conditions.  

An implicit Laasonen solver is used.  PyTorch runs it on the GPU.  
**TODO** - See if a MacCormack solver conserves area under the probability
curve better.

A wave packet is launched toward a potential barrier, which splits the
packet into a left-moving and a right-moving packet.  Upon meeting up,
the two wave packets produce an interference pattern.  

In [ ]:
import torch
import pycuda.driver as cuda
import pycuda.autoinit
import matplotlib.pyplot as plt

import holoviews as hv
from holoviews import opts
from holoviews.streams import Pipe, Buffer
import numpy as np
import time, math

hv.extension('bokeh')

cuda.init()

### A function to print available GPU memory

In [ ]:
def print_memory_info(msg=None):
    
    #print("torch.cuda.memory_allocated:    %fGB"%(torch.cuda.memory_allocated(0)/1024/1024/1024))
    #print("torch.cuda.memory_reserved:     %fGB"%(torch.cuda.memory_reserved(0)/1024/1024/1024))
    #print("torch.cuda.max_memory_reserved: %fGB"%(torch.cuda.max_memory_reserved(0)/1024/1024/1024))

    availableBytes, totalBytes = cuda.mem_get_info()
    if msg is None:
        print(f'GPU has {availableBytes/1e9:.2f} GB available of {totalBytes/1e9:.2f} GB total')
    else:
        print(f'{msg}: GPU has {availableBytes/1e9:.2f} GB available of {totalBytes/1e9:.2f} GB total')

print_memory_info()

### Set some constants

In [ ]:
# The imaginary number i
IMAG_NUM = torch.complex(torch.zeros(1, dtype=torch.float32), torch.ones(1, dtype=torch.float32))
IMAG_NUM

In [ ]:
IDIM = 2048
XMIN = 0
XMAX = 20
X_INIT_PACKET_CTR = 0.5 * (XMIN + XMAX)
LAMBDA0 = 1
MASS = 1

In [ ]:
dx = (XMAX-XMIN)/(IDIM-1)
dt = 0.005 * dx

a = torch.complex( (0.5 * dt/MASS/dx**2) * torch.ones(1, dtype=torch.float32), torch.zeros(1, dtype=torch.float32))
b = IMAG_NUM - 2*a

x_arry = np.arange(XMIN, XMAX+dx, dx)

### Plotting functions

In [ ]:
def plot_psi(x_arry, psi, step):
    
    plt.plot(x_arry, psi.real, c='g', label='real')
    plt.plot(x_arry, psi.imag, c='r', label='imaginary')
    plt.xlabel('x')
    plt.ylabel('Ψ')
    plt.grid('both')
    plt.legend()
    plt.title(f'Wave function Ψ at time step {step}')
    plt.show()

def plot_psi_star_psi(x_array, psi, step):
    
    psi_star = torch.conj(psi)
    psi_star_diag = torch.diag(psi_star)
    psi_star_psi = psi_star_diag.matmul(psi)
    psi_star_psi = psi_star_psi.real  # imaginary part is zero
    
    plt.plot(x_arry, psi_star_psi)
    plt.xlabel('x')
    plt.ylabel('Ψ∗Ψ')
    plt.grid('both')
    plt.title(f'Ψ∗Ψ at time step {step}')
    plt.show() 

### Function to initialize the right-moving wave packet

In [ ]:
def create_initial_wavepacket(x_arry, x_packet_center, lambda0):
    
    two_over_pi = 2.0/np.pi
    two_pi_over_lambda0 = 2.0 * np.pi / lambda0
    
    idim = len(x_arry)
    
    psi_re = torch.zeros(idim, dtype=torch.float32)
    psi_im = torch.zeros(idim, dtype=torch.float32)
    
    for i,x in enumerate(x_arry):
        x_minus_x_packet_center = x - x_packet_center
        amplitude = (two_over_pi)**0.25 * np.exp(-x_minus_x_packet_center**2)
        psi_re[i] = amplitude * np.cos(two_pi_over_lambda0 * x)
        psi_im[i] = amplitude * np.sin(two_pi_over_lambda0 * x)
    
    psi = torch.complex(psi_re, psi_im)
    
    print(f'Created psi across idim = {len(psi)} grid points')
    
    return psi

### Initialize the tri-diagonal matrix of the left hand side of the implicit finite difference equation
Then find its inverse.  

In [ ]:
Lr = torch.zeros(IDIM, IDIM)
Lc = torch.zeros(IDIM, IDIM)

L = torch.complex(Lr, Lc)

for i in range(IDIM):
    L[i,i] = b
    for j in range(i-1, i+2, 2):
        if 0 <= j and j < IDIM:
            L[i,j] = a

L[0, IDIM-2] = a
L[IDIM-1, 1] = a

Linv = torch.inverse(L)

### Define the potential barrier
It's the V(x) in the Schroedinger equation.  

In [ ]:
V = torch.zeros(IDIM)
VMAX = 20

i_pot_step_start = int(0.75 * IDIM)
num_steps        = int(0.15*IDIM)

for i in range(num_steps):
    i_step = i_pot_step_start + i
    frac = i/num_steps
    V[i_step] = VMAX - frac*VMAX
    #print(f'i={i_step}, V={V[i_step]}')

In [ ]:
V = torch.zeros(IDIM)
VMAX = 100

i_pot_step_start = int(0.75 * IDIM)
num_steps        = 6

for i in range(num_steps):
    i_step = i_pot_step_start + i
    V[i_step] = VMAX

### Plot the initial Ψ, the wave function
It's a wave packet that will move to the right.

In [ ]:
psi = create_initial_wavepacket(x_arry, X_INIT_PACKET_CTR, LAMBDA0)

plot_psi(x_arry, psi, 0)
plot_psi_star_psi(x_arry, psi, 0)

### Move some PyTorch tensors into the GPU
Print GPU memory before and after moving tensors over.  

In [ ]:
print_memory_info('Before allocating GPU memory')

IMAG_NUM_cuda = IMAG_NUM.cuda()
print_memory_info('After IMAG_NUM')

psi_cuda      = psi.cuda()
print_memory_info('After psi_cuda')

Linv_cuda     = Linv.cuda()
print_memory_info('After Linv_cuda')

V_cuda        = V.cuda()
print_memory_info('After V_cuda')

### Define the function that solves the Schroedinger equation for Ψ(x,t)

In [ ]:
def solve(pipe, x_arry, psi):
    
    # pipe      ...a Holoviews stream to push updated Ψ(x) to
    # x_array   ...the x grid
    # psi       ...Ψ(x)
    
    print_memory_info('Right before copying tensors to the GPU')
    
    IMAG_NUM_cuda = IMAG_NUM.cuda()
    psi_cuda      = psi.cuda()
    Linv_cuda     = Linv.cuda()
    V_cuda        = V.cuda()
    
    #Extras = []
    #for k in range(128):
    #    Linv_cuda1 = Linv.cuda()
    #    Extras.append(Linv_cuda1)
    #    print_memory_info(f'After Linv_cuda1 for k = {k}')
    
    print_memory_info('Right after copying tensors to the GPU')
    
    total_steps = 0
    count_til_plot = 1
    area = 1.0
    while(area > 0.95):
        psi_cuda = (IMAG_NUM_cuda + V_cuda*dt) * psi_cuda
        psi_cuda = Linv_cuda.matmul(psi_cuda)
        total_steps += 1

        count_til_plot -= 1
        if count_til_plot == 0:          
            psi = psi_cuda.cpu()
            psi_star = torch.conj(psi)
            psi_star_diag = torch.diag(psi_star)
            psi_star_psi = psi_star_diag.matmul(psi)
            psi_star_psi = psi_star_psi.real  # imaginary part is zero
            
            pipe.send( {'x': x_arry,'y': psi_star_psi} )
            count_til_plot = 50
        
        # keep the area 1.0
        area = np.trapz(x=x_arry, y=psi_star_psi)
        #print(f'Step {total_steps}: area = {area}')
        #psi = psi / math.sqrt(area)
        #psi_cuda = psi.cuda()
        
        #time.sleep(0.1)
    
    print(f'STOPPED at step {total_steps}: area = {area}')
    print_memory_info()

# Solve the Schroedinger equation
**Update the plot of Ψ as the calculation runs.**

Uses Holoviews with a stream.

https://medium.com/@Georgi.Ganchev/dynamic-plots-in-python-jupyter-notebooks-holoviews-f94626de2291
https://holoviews.org/user_guide/Streaming_Data.html

In [ ]:
pipe = Pipe(data=[])
dynamic_plot = hv.DynamicMap(hv.Curve, streams=[pipe])
dynamic_plot.opts(opts.Curve(width=950, show_grid=True, xlim=(XMIN, XMAX), ylim=(0, 1)))

# PLOT WILL APPEAR BELOW ↓

In [ ]:
# PLOT WILL APPEAR ABOVE ↑

# initialize Ψ, then Ψ*Ψ

psi           = create_initial_wavepacket(x_arry, X_INIT_PACKET_CTR, LAMBDA0)
psi_star      = torch.conj(psi)
psi_star_diag = torch.diag(psi_star)
psi_star_psi  = psi_star_diag.matmul(psi)
psi_star_psi  = psi_star_psi.real  # imaginary part is zero

# compute the area under Ψ*Ψ (should be 1.0)

area = np.trapz(x=x_arry, y=psi_star_psi)
print(f'Initial area under Ψ*Ψ = {area:.4f}   ...should be 1.0') 

# stream the initial Ψ for plotting

pipe.send( {'x': np.array(x_arry), 'y': np.array(psi_star_psi)} )

# solve Ψ over many time steps

solve(pipe, x_arry, psi)